In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.layers import LSTM,Dropout,GRU,MaxPooling1D,Flatten,concatenate
from keras.models import Sequential
from keras.layers import Dense,Concatenate,Bidirectional
from keras.layers.convolutional import Conv1D
import tensorflow as tf
from keras.utils.vis_utils import model_to_dot
from keras.layers import RepeatVector
from keras.layers import TimeDistributed,Activation
from sklearn.preprocessing import MinMaxScaler
from keras import Input,Model
FD1_train = pd.read_csv("train_FD001.txt",delimiter=" ",header = None)
FD1_test = pd.read_csv("test_FD001.txt",delimiter=" ",header = None)
RUL_file=pd.read_csv("RUL_FD001.txt",header = None)
FD1_test=FD1_test.drop([26, 27], axis = 1)
columns = {0:'unit_number',1:'time_in_cycles',2:'opSetting1',3:'opSetting2',4:'opSetting3',5:'sensor1',6:'sensor2',
           7:'sensor3',8:'sensor4',9:'sensor5',10:'sensor6',11:'sensor7',12:'sensor8',13:'sensor9',14:'sensor10',
           15:'sensor11',16:'sensor12',17:'sensor13',18:'sensor14',19:'sensor15',20:'sensor16',
           21:'sensor17',22:'sensor18',23:'sensor19',24:'sensor20',25:'sensor21'}
FD1_test.columns=columns
FD1_test = FD1_test.rename(columns=columns)
#FD1_test=FD1_test.drop(columns = ['opSetting1','opSetting2','opSetting3','sensor1','sensor5','sensor6','sensor10','sensor16','sensor18','sensor19','sensor17'], axis =1)

FD1_test.drop(FD1_test.loc[FD1_test['unit_number']==1].index, inplace=True)
FD1_test.drop(FD1_test.loc[FD1_test['unit_number']==2].index, inplace=True)
FD1_test.drop(FD1_test.loc[FD1_test['unit_number']==14].index, inplace=True)
FD1_test.drop(FD1_test.loc[FD1_test['unit_number']==22].index, inplace=True)
FD1_test.drop(FD1_test.loc[FD1_test['unit_number']==25].index, inplace=True)
FD1_test.drop(FD1_test.loc[FD1_test['unit_number']==39].index, inplace=True)
FD1_test.drop(FD1_test.loc[FD1_test['unit_number']==85].index, inplace=True)

cols_normalize = FD1_test.columns.difference(
    ['unit_number', 'time_in_cycles'])  

min_max_scaler = MinMaxScaler()

norm_train_df = pd.DataFrame(min_max_scaler.fit_transform(FD1_test[cols_normalize]),
                             columns=cols_normalize,
                             index=FD1_test.index)

join_df = FD1_test[FD1_test.columns.difference(cols_normalize)].join(norm_train_df)
FD1_test = join_df.reindex(columns=FD1_test.columns)
#FD1_test=FD1_test.drop(columns = ['time_in_cycles'], axis =1)
data_cols= FD1_test.columns

def prepare_test_data(df):
    data_list = []
    for unit_number in df.unit_number.unique():
        unit = df[df.unit_number == unit_number]
        data_list.append(np.array(unit[data_cols])[-50:, :])
    return np.stack(data_list)
FD1_test=prepare_test_data(FD1_test)
FD1_test=FD1_test[:,:,1:]






FD1_train=FD1_train.drop([26, 27], axis = 1)
columns = {0:'unit_number',1:'time_in_cycles',2:'opSetting1',3:'opSetting2',4:'opSetting3',5:'sensor1',6:'sensor2',
           7:'sensor3',8:'sensor4',9:'sensor5',10:'sensor6',11:'sensor7',12:'sensor8',13:'sensor9',14:'sensor10',
           15:'sensor11',16:'sensor12',17:'sensor13',18:'sensor14',19:'sensor15',20:'sensor16',
           21:'sensor17',22:'sensor18',23:'sensor19',24:'sensor20',25:'sensor21'}
FD1_train = FD1_train.rename(columns=columns)
#FD1_train=FD1_train.drop(columns = ['opSetting1','opSetting2','opSetting3','sensor1','sensor5','sensor6','sensor10','sensor16','sensor18','sensor19','sensor17'], axis =1)
def Calcolo_RUL(df):
    df_copy=df.copy()
    gruppo=df_copy.groupby('unit_number')['time_in_cycles'].max().reset_index()
    gruppo.columns = ['unit_number','max_cycles_fu']
    df_copy = df_copy.merge(gruppo, on=['unit_number'], how='left')
    df['RUL'] = df_copy['max_cycles_fu'] - df_copy['time_in_cycles']
    return df
Calcolo_RUL(FD1_train)
rul_clip_limit = 130
FD1_train=FD1_train.clip(upper=rul_clip_limit)



cols_normalize = FD1_train.columns.difference(
    ['unit_number', 'time_in_cycles', 'RUL'])  # NORMALIZE COLUMNS except [id , cycle, rul ....]

min_max_scaler = MinMaxScaler()

norm_train_df = pd.DataFrame(min_max_scaler.fit_transform(FD1_train[cols_normalize]),
                             columns=cols_normalize,
                             index=FD1_train.index)

join_df = FD1_train[FD1_train.columns.difference(cols_normalize)].join(norm_train_df)
FD1_train = join_df.reindex(columns=FD1_train.columns)

FD1_c=FD1_train.copy()
FD1_train = FD1_train.drop(['RUL'], axis=1)

def gen_sequence(id_df, seq_length, seq_cols):
    data_array = id_df[seq_cols].values
    num_elements = data_array.shape[0]
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        yield data_array[start:stop, :]

seq_gen = (list(gen_sequence(FD1_train[FD1_train['unit_number']==unit_number],50, FD1_train.columns))
           for unit_number in FD1_train['unit_number'].unique())

seq_array = np.concatenate(list(seq_gen)).astype(np.float32)
seq_array=seq_array[:,:,1:]
seq_array1=seq_array[:7815,:,:]
seq_array2=seq_array[7816:,:,:]


def gen_labels(id_df, seq_length, label):
    data_array = id_df[label].values
    num_elements = data_array.shape[0]
    return data_array[seq_length:num_elements, :]

label_gen = [gen_labels(FD1_c[FD1_c['unit_number']==id], 50, ['RUL'])
             for id in FD1_c['unit_number'].unique()]
label_array = np.concatenate(label_gen).astype(np.float32)



a=Input(shape=(50,25))
x1=Conv1D(128,1,padding='same',activation='relu')(a)
x1=MaxPooling1D(pool_size=2,padding='same',strides=2)(x1)
x1=Conv1D(64,1,padding='same',activation='relu')(x1)
x1=MaxPooling1D(pool_size=2,padding='same',strides=2)(x1)
x1=Conv1D(32,1,padding='same',activation='relu')(x1)
x1=MaxPooling1D(pool_size=2,padding='same',strides=2)(x1)
x1=Dropout(0.2)(x1)
x1 = Model(inputs=a, outputs=x1)

b=Input(shape=(50,25))
y1=LSTM(128, return_sequences=True)(b)
y1=LSTM(64, return_sequences=True)(y1)
y1=LSTM(32, return_sequences=True)(y1)
y1 = Model(inputs=b, outputs=y1)

combined = concatenate([x1.output, y1.output],axis=0)
z=Conv1D(128,1,activation='relu')(combined)
z=MaxPooling1D(pool_size=2,padding='same',strides=2)(z)
z= Flatten()(z)
z = Dense(128,activation="relu")(z)
z = Dense(32,activation="relu")(z)
z = Dense(1,activation="relu")(z)

model = Model(inputs=[y1.input, x1.input], outputs=z)
model.compile(loss='mean_squared_error', optimizer='adam',metrics = [tf.keras.metrics.RootMeanSquaredError()])
model.fit([seq_array1,seq_array2],label_array,batch_size=90,epochs=20,verbose=1,validation_split=0.05)
y_pred=model.predict([FD1_test,FD1_test])
from sklearn.metrics import mean_squared_error
from math import sqrt
#from sklearn.metrics import mean_absolute_percentage_error
RUL_file.drop([1,2,14,22,25,39,85], inplace=True)

print(sqrt(mean_squared_error(RUL_file, y_pred)))
print(y_pred)
print((mean_squared_error(RUL_file, y_pred)))


ValueError: ignored